Example of pytorch + LSTM

# Imports and settings

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
# from tqdm import trange

# from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# Obtain data

Prediction will be made for the Dow Jones Industrial Average stock market index.

Data taken from the open dataset:
https://www.openml.org/search?type=data&status=active&id=43840

In [ ]:
from sklearn.datasets import fetch_openml

# fetch raw data
stock_data = fetch_openml(data_id=43840, parser='pandas').data
stock_data.head()

In [ ]:
# convert data types and drop unnecessary columns

# convert string dates
stock_data['Date'] = stock_data['Date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y'))

# convert string comma-separated floats
stock_data[['Price', 'Open', 'High', 'Low']] = stock_data[['Price', 'Open', 'High', 'Low']].map(lambda x: float(x.replace(',', '')))

# drop other columns
stock_data = stock_data.drop(columns=['Vol.', 'Change_%'])

# convert Date to index
stock_data = stock_data.set_index('Date')

stock_data.head()

In [ ]:
dt_start = stock_data.index.min()
dt_end = stock_data.index.max()

In [ ]:
# for some days, there is no data
# impute the last observable close prices for those days

business_days = pd.date_range(dt_start, dt_end, freq='B')

data = (
    stock_data # take close prices
    .reindex(business_days) # drop holidays and weekends, if any, place NaN for missing business days
    .fillna(method='ffill') # replace NaN with the last observable price
    .dropna() # ensure that there are no NaN left in the dataset
)

data.head()

In [ ]:
# plot prices

fig = plt.figure(figsize=(10,6))
sns.lineplot(data)

# Predict Price with LSTM

## configuring pytorch

In [ ]:
import torch

# fix random seed
SEED = 0

# set precision
torch.set_default_dtype(torch.float32)

# set device
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

## train-test split and scaling data

In [ ]:
from sklearn.preprocessing import RobustScaler

# train-test split
X_train = data[data.index < '2019-01-01']
X_test = data[data.index >= '2019-01-01']

dates_test = X_test.index

# scale X
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# find index of the Price column since X_train is a numpy array
price_ind = list(data.columns).index('Price')

In [ ]:
# how much history is used at each iteration
history_days = 30

# model will predict Price at T + hrz, where T is the last date in train
hrz = 7

def prepare_data_for_lstm(X):
  # Assume that history_days = L = 30 and hrz = 7,
  # X is T x n_dim array of series we want to predict, where each column is a series
  #
  # For LSTM, the input series data X is split to a batch of sequences
  #     X_0, X_1, ..., X_29 -> X_36
  #     X_1, X_2, ..., X_30 -> X_37
  #     ...
  #     X_(T-37), X_(T-36), ..., X_(T-8) -> X_(T-1)
  # The whole sequence X_0, ..., X_(T-8) is split a batch so only the last
  # 30 days influence the prediction.
  #
  # Let N = T-L-hrz+1 - batch length
  #
  # Returns
  #     [seq_0, ..., seq_(T-L-hrz)], where seq_k= [X_k, ..., X_{k+L-1}],
  #         seq_k is (L, n_dim), n_dim is a number of predicted series.
  #         Total size: (N, L, n_dim)
  #     [X_{L+hrz-1}, ..., X_{T-1}] - target for prediction.
  #         Total size: (N, n_dim)
  #

  X = np.asarray(X)

  X_prep = np.stack([X[i-history_days:i,:] for i in range(history_days, X.shape[0]-hrz)])
  y_prep = np.stack([X[i+hrz,:] for i in range(history_days, X.shape[0]-hrz)])

  print('X_prep: ', X_prep.shape)
  print('y_prep: ', y_prep.shape)

  return torch.tensor(X_prep, dtype=torch.float32, device=DEVICE), torch.tensor(y_prep, dtype=torch.float32, device=DEVICE)

X_train_lstm, y_train_lstm = prepare_data_for_lstm(X_train)
X_test_lstm, y_test_lstm = prepare_data_for_lstm(X_test)

In [ ]:
# sequential NN model with LSTM block:
#     LSTM -> Dropout -> Linear

activation = torch.nn.LeakyReLU

class MyLSTM(torch.nn.Module):
    def __init__(self, input_size, output_size, lstm_hidden_size, lstm_num_layers=1):
        super().__init__()

        self.my_lstm = torch.nn.LSTM(input_size, lstm_hidden_size, num_layers=lstm_num_layers, batch_first=True)

        self.final_stage = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
          torch.nn.Linear(lstm_hidden_size, output_size)
        )

    def forward(self, x):
        """
        Parameters
        ----------
        x : torch.Tensor
            of size : batch_size x n_input

        Returns
        ----------
        torch.Tensor
            of size : batch_size x n_output

        """
        res, _ = self.my_lstm(x) # size (N, L, lstm_hidden_size)

        # get only the last prediction for each sequence
        res = res[:,-1,:] # size (N, lstm_hidden_size)

        return self.final_stage(res)

In [ ]:
model = MyLSTM(
    input_size=X_train_lstm.shape[2],
    output_size=1,
    lstm_hidden_size=100*X_train_lstm.shape[2],
    lstm_num_layers=1
    )

if torch.cuda.is_available():
    model.cuda()

In [ ]:
# split to 2 batches during training
# parameters are updeted after each batch
batch_size = X_train_lstm.shape[0] // 2 + 1

# indices of predicted series.
# Only Price is required in this example.
pred_ind = [price_ind]

loss_f = torch.nn.MSELoss() # validation loss
loss_f_train = loss_f # train loss

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=1e-6)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=(200), gamma=0.3)
scheduler = None

loss_dynamics_train = []
loss_dynamics_val = []

loss_train_prev = np.nan

print('-----------------------------------------------------------')
print('| epoch | training_loss | loss decrease | validation loss |')
print('-----------------------------------------------------------')

for epoch in range(2000):
    loss_batch_train = []
    loss_batch_val = []
    for X, y in zip(torch.split(X_train_lstm, batch_size), torch.split(y_train_lstm, batch_size)):

        y = y[:,pred_ind]
        model.train(True)

        y_pred = model(X)
        loss = loss_f_train(y, y_pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.train(False)

        y_pred = model(X)
        loss_batch_train.append(torch.sqrt(loss_f(y, y_pred)).cpu().detach().numpy() + 0.0)

        y_pred = model(X_test_lstm)
        loss_batch_val.append(torch.sqrt(loss_f(y_test_lstm[:,pred_ind], y_pred)).cpu().detach().numpy() + 0.0)

    loss_epoch_train = np.mean(loss_batch_train)
    loss_dynamics_train.append(loss_epoch_train)

    loss_epoch_val = np.mean(loss_batch_val)
    loss_dynamics_val.append(loss_epoch_val)

    s = '| {0:5d} | {1:13.6f} | {2:12.6f}% | {3:15.6f} |'.format(
        epoch,
        loss_epoch_train,
        (loss_epoch_train - loss_train_prev) / (1e-12 + loss_train_prev) * 100,
        loss_epoch_val
    )
    if epoch % 100 == 0:
        print(s)

    loss_train_prev = loss_epoch_train

    if scheduler is not None:
        scheduler.step()


In [ ]:
y_true = scaler.inverse_transform(y_test_lstm.cpu().detach().numpy())[:,price_ind]

# to apply the inverse transform, tile model prediction to match the shape of X
y_pred_for_scale = torch.tile(model(X_test_lstm), (1,X_train_lstm.shape[2]))

# transform and select only the Price column
y_pred = scaler.inverse_transform(y_pred_for_scale.cpu().detach().numpy())[:,price_ind]

In [ ]:
# plot results

plt.plot(dates_test[history_days:-hrz], y_true, '-k', label='Price')
plt.plot(dates_test[history_days:-hrz], y_pred, '--r', label='Forecast 7 day')

plt.title('Price')
plt.xticks(rotation=70)
plt.legend()

plt.show()